In [1]:
import math
import numpy as np

In [2]:
EPS = 1e-7
ITERATIONS = 200000

In [3]:
def _format_matrix(min, max, size):
    return np.array((max - min) * np.random.random((size, size)) + min, dtype=np.float64)
    
def rand_matrix(size):
    A = np.array(np.random.rand(size, size), dtype=np.float64)
    A = np.tril(A) + np.tril(A, -1).T
    return A.T * A

def diag_matrix(size):
    A = _format_matrix(0.0, 10.0, size)
    for i in range(size):
        A[i, i] = sum(A[i]) + 1
    return A

def hilbert_matrix(size):
    ans = np.array(np.zeros((size, size)), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            ans[i, j] = 1.0 / (i + j + 1)
    return ans

def gen_right_column(size):
    return np.array(np.random.random(size), dtype=np.float64)

In [4]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [5]:
def zeidelMethod(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            
    print("Iterations = ", curIterations)
    return xn

In [6]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    Relax = 1.6
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            
    print("Iterations = ", curIterations)
    return xn

In [7]:
def _test(method, matrix, column):
    return method(matrix, column)

def _check_eps(a, b, eps):
    return abs(a - b) < eps
EPSILON = 1e-2
def _check_ans(matrix, column, res, size):
    eps = EPSILON
    c = np.matmul(matrix, res)
    for i in range(size):
        if not _check_eps(c[i], column[i], eps):
            return False
    return True

def _testAll(matrix_gen, size, methods):
    matrix, column = matrix_gen(size), gen_right_column(size)
    print(f'Тест для матрицы {matrix} и столбца {column}')
    for method in methods:
        print(f'Результат выполнения метода {method.__name__}:')
        res = _test(method, matrix, column)
        check = _check_ans(matrix, column, res, size)
        if check:
            print(f'{res}. Погрешность не больше {EPSILON}\n')
        else:
            print(f'разошлось: matrix * res = {np.matmul(matrix, res)}\n ожидаемый столбец = {column}\n')

def _get_methods():
    return [zeidelMethod, zeidelMethodRelax]

def testDiagMatrix(size):
    _testAll(diag_matrix, size, _get_methods())

def testHilbertMatrix(size):
    _testAll(hilbert_matrix, size, _get_methods())
    
def testRandMatrix(size):
    _testAll(rand_matrix, size, _get_methods())

In [8]:
testDiagMatrix(10)

Тест для матрицы [[4.08760255e+01 8.26978890e+00 8.07176806e+00 6.07554151e-01
  1.18323111e+00 7.15186074e+00 3.21567990e+00 4.01315884e+00
  3.46235169e+00 1.17948910e+00]
 [4.70262849e+00 3.44575606e+01 7.15443311e+00 5.59789103e+00
  3.32647471e+00 1.85521683e+00 2.57533979e-03 3.65480048e+00
  6.36211201e+00 5.73003088e-01]
 [7.38959659e+00 1.89208856e+00 3.99982598e+01 8.42079719e-01
  9.13801729e+00 1.91418146e+00 1.09963947e+00 8.38365433e+00
  2.73336705e+00 3.45674189e-01]
 [5.38661680e+00 4.79845276e+00 8.27642891e-01 2.79021026e+01
  2.85169216e+00 1.05339798e+00 1.22620793e+00 1.83663490e+00
  1.75854249e+00 2.27269405e+00]
 [1.98196070e+00 3.67062501e+00 8.45754980e-01 9.70919529e+00
  5.40035453e+01 3.62268756e-01 9.01479236e+00 9.55150876e+00
  6.58401441e+00 2.30777867e+00]
 [6.07325755e+00 4.78052108e+00 1.59554746e-01 5.44010246e+00
  9.80636023e+00 5.41850413e+01 9.83132175e+00 1.39135077e+00
  8.91043995e+00 4.61195379e+00]
 [3.09270780e+00 4.63918514e-01 1.2114271

In [9]:
testRandMatrix(4)

Тест для матрицы [[0.05835427 0.73027824 0.28837618 0.65158864]
 [0.73027824 0.21110305 0.26038914 0.60097907]
 [0.28837618 0.26038914 0.43614655 0.39303548]
 [0.65158864 0.60097907 0.39303548 0.71534538]] и столбца [0.47560132 0.86607027 0.87276787 0.32746265]
Результат выполнения метода zeidelMethod:


/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Iterations =  200000
разошлось: matrix * res = [nan nan nan nan]
 ожидаемый столбец = [0.47560132 0.86607027 0.87276787 0.32746265]

Результат выполнения метода zeidelMethodRelax:


/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in double_scalars
  if sys.path[0] == '':


Iterations =  200000
разошлось: matrix * res = [nan nan nan nan]
 ожидаемый столбец = [0.47560132 0.86607027 0.87276787 0.32746265]



In [10]:
testHilbertMatrix(5)

Тест для матрицы [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]] и столбца [0.60878608 0.27619191 0.86041168 0.03678769 0.35381665]
Результат выполнения метода zeidelMethod:
Iterations =  200000
[1006.9817309019854, -18583.997568597555, 79437.08669667656, -119201.85398486856, 58035.30213863386]. Погрешность не больше 0.01

Результат выполнения метода zeidelMethodRelax:
Iterations =  110640
[1007.1960690404424, -18587.932138891003, 79453.84936268497, -119226.95033629498, 58047.499581639466]. Погрешность не больше 0.01

